**DECISION TREE**

Importing libraries

In [12]:
import torch
import tensorflow as tf
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from dataset import TimeSeriesDataset
from dataset import NumpyToTensor
import sklearn
from sklearn.tree import DecisionTreeRegressor
import math
from sklearn.metrics import mean_squared_error

Importing data

In [13]:
train_dataa = pd.read_csv("data/train_delhi_aqi.csv")
test_dataa = pd.read_csv("data/test_delhi_aqi.csv")

In [14]:
# need to be tuned:
sequence_size = 100
num_levels = 6
min_samples_split = 1/(12*13*100)
min_samples_leaf = 1/(12*13*1000)

Setting up dataloader

In [15]:
train_data = TimeSeriesDataset(train_dataa,sequence_size)
# train_dataloader = DataLoader(dataset=train_data, batch_size = math.ceil(len(train_data)/(12*13)), shuffle=False, num_workers=2)
train_dataloader = DataLoader(dataset=train_data, batch_size = 10000, shuffle=False, num_workers=2)

Converting the dataloader into an iterable

In [16]:
dataiter = iter(train_dataloader)

In [17]:
data = next(dataiter)

Creating the tree classes

In [18]:
# # Fit regression model
# regr_1 = DecisionTreeRegressor(max_depth=2)
# regr_2 = DecisionTreeRegressor(max_depth=5)
# regr_1.fit(X, y)
# regr_2.fit(X, y)

# # Predict
# X_test = np.arange(0.0, 5.0, 0.01)[:, np.newaxis]
# y_1 = regr_1.predict(X_test)
# y_2 = regr_2.predict(X_test)


In [26]:
tree6 = DecisionTreeRegressor(max_depth=num_levels, criterion='squared_error', min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf)

tree2= DecisionTreeRegressor(max_depth=2, criterion='squared_error', min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf)

tree5= DecisionTreeRegressor(max_depth=5, criterion='squared_error', min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf)

tree4= DecisionTreeRegressor(max_depth=4, criterion='squared_error', min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf)

tree7= DecisionTreeRegressor(max_depth=7, criterion='squared_error', min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf)

Training Loop

In [48]:
epochs = 2
size = len(train_data)
niters = math.ceil(size/100)
print(size, niters)

2516454 25165


In [55]:
for ep in range(epochs):
    for i, (x, y) in enumerate(train_dataloader):
        tree2.fit(x,y)
        tree4.fit(x,y)
        tree5.fit(x,y)
        tree6.fit(x,y)
        tree7.fit(x,y)
        if i%10==0:
            ypred6 = tree6.predict(x)
            ypred5 = tree5.predict(x)
            ypred7 = tree7.predict(x)
            ypred4 = tree4.predict(x)
            ypred2 = tree2.predict(x)
            err6 = mean_squared_error(y,ypred6)
            err7 = mean_squared_error(y,ypred7)
            err2 = mean_squared_error(y,ypred2)
            err4 = mean_squared_error(y,ypred4)
            err5 = mean_squared_error(y,ypred5)
            print(f'Epoch {ep}, Iteration {i}, tree 2, train error: {err2}')
            print(f'Epoch {ep}, Iteration {i}, tree 4, train error: {err4}')
            print(f'Epoch {ep}, Iteration {i}, tree 5, train error: {err5}')
            print(f'Epoch {ep}, Iteration {i}, tree 6, train error: {err6}')
            print(f'Epoch {ep}, Iteration {i}, tree 7, train error: {err7}')
        if i > 100:
            break
    print(f'epoch {ep}/{epochs} completed')
    break

Epoch 0, Iteration 0, tree 2, train error: 691.0803844810924
Epoch 0, Iteration 0, tree 4, train error: 54.537557542311816
Epoch 0, Iteration 0, tree 5, train error: 14.344612811273587
Epoch 0, Iteration 0, tree 6, train error: 3.278938831675475
Epoch 0, Iteration 0, tree 7, train error: 0.7730672576441379
Epoch 0, Iteration 10, tree 2, train error: 685.6846017657958
Epoch 0, Iteration 10, tree 4, train error: 55.567433631680814
Epoch 0, Iteration 10, tree 5, train error: 13.80212893767701
Epoch 0, Iteration 10, tree 6, train error: 3.313428091970515
Epoch 0, Iteration 10, tree 7, train error: 0.7993960727705467
Epoch 0, Iteration 20, tree 2, train error: 700.9688667533457
Epoch 0, Iteration 20, tree 4, train error: 57.76447022193678
Epoch 0, Iteration 20, tree 5, train error: 15.118032612776357
Epoch 0, Iteration 20, tree 6, train error: 3.5716127014902246
Epoch 0, Iteration 20, tree 7, train error: 0.8910071097184052
Epoch 0, Iteration 30, tree 2, train error: 700.8818575570947
Epoch

In [56]:
i

101

In [57]:
test_data = TimeSeriesDataset(test_dataa,sequence_size)
test_dataloader = DataLoader(dataset=test_data, batch_size = 1, shuffle=False, num_workers = 3)
len(test_data)

139710

In [58]:
#calc error
errs2 = torch.empty((len(test_data),1))
errs4 = torch.empty((len(test_data),1))
errs5 = torch.empty((len(test_data),1))
errs6 = torch.empty((len(test_data),1))
errs7 = torch.empty((len(test_data),1))

for ep in range(1):
    for i, (x,y) in enumerate(test_dataloader):
        ypred6 = tree6.predict(x)
        ypred5 = tree5.predict(x)
        ypred7 = tree7.predict(x)
        ypred4 = tree4.predict(x)
        ypred2 = tree2.predict(x)
        err6 = mean_squared_error(y,ypred6)
        err7 = mean_squared_error(y,ypred7)
        err2 = mean_squared_error(y,ypred2)
        err4 = mean_squared_error(y,ypred4)
        err5 = mean_squared_error(y,ypred5)
        errs2[i] = err2
        errs4[i] = err4
        errs5[i] = err5
        errs6[i] = err6
        errs7[i] = err7

In [59]:
i

139709

In [60]:
print(f'Tree2 max error: {torch.max(errs2)}, min error: {torch.min(errs2)}')
print(f'Tree4 max error: {torch.max(errs4)}, min error: {torch.min(errs4)}')
print(f'Tree5 max error: {torch.max(errs5)}, min error: {torch.min(errs5)}')
print(f'Tree6 max error: {torch.max(errs6)}, min error: {torch.min(errs6)}')
print(f'Tree7 max error: {torch.max(errs7)}, min error: {torch.min(errs7)}')

Tree2 max error: 7720.57861328125, min error: 1.8322680261917412e-05
Tree4 max error: 717.2456665039062, min error: 3.8890334508323576e-07
Tree5 max error: 169.72816467285156, min error: 9.344344675810135e-08
Tree6 max error: 90.09158325195312, min error: 0.0
Tree7 max error: 81.75154876708984, min error: 0.0


Plotting the tree

In [42]:
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree

In [43]:
tree6

DecisionTreeRegressor(max_depth=6, min_samples_leaf=6.41025641025641e-06,
                      min_samples_split=6.41025641025641e-05)

In [44]:
tree7

DecisionTreeRegressor(max_depth=7, min_samples_leaf=6.41025641025641e-06,
                      min_samples_split=6.41025641025641e-05)

In [45]:
min_samples_split*len(train_data)

161.31115384615384

In [47]:
min_samples_leaf*len(train_data)

16.131115384615384